In [16]:
! pip install mediapipe

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached jax-0.4.35-py3-none-any.whl.metadata (22 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 28.1 MB/s eta 0:00:0000:0100:01
Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (107 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached jax-0.4.35-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 36.6 MB/s eta 0:00:00a 0:00:01
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl (63.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.2/750.2 kB 26.7 MB/s eta 0:00:00


In [17]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import pandas as pd
import csv
import os

# Load Model

In [21]:
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='../face_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE)

face_mesh_connections = mp.solutions.face_mesh.FACEMESH_TESSELATION

In [5]:
DIRECTORY = r"../DATASET/train"
CATEGORIES = []


try:
    folders = os.listdir(DIRECTORY)
    print(f"Directories in '{DIRECTORY}':")
    for folder in folders:
        if os.path.isdir(os.path.join(DIRECTORY, folder)):
            CATEGORIES.append(folder)
except ValueError as e:
    print(e)

print("Categories:", CATEGORIES)

Directories in '../DATASET/train':
Categories: ['7', '6', '1', '4', '3', '2', '5']


In [6]:
face_mesh_connections = mp.solutions.face_mesh.FACEMESH_TESSELATION
folder_path = '../output_data'  # 您希望創建的資料夾名稱
file_path = os.path.join(folder_path, 'connection.csv')
os.makedirs(folder_path, exist_ok=True)
with open(file_path, 'w', newline='') as connections_csv:
    connections_writer = csv.writer(connections_csv)
    connections_writer.writerow(["index","point1", "point2"])
    index = -1
    for connection in face_mesh_connections:
        index += 1
        point1 = connection[0]
        point2 = connection[1]
        connections_writer.writerow([index, point1, point2])

# Get file

In [15]:
# 指定處理類別
desired_category = '6'

def paths(desired_category):
    # Check if the category directory exists
    category_path = os.path.join(DIRECTORY, desired_category)

    if not os.path.isdir(category_path):
        print(f"Category '{desired_category}' does not exist in '{DIRECTORY}'.")
        # List available categories if the desired one is missing
        available_categories = [folder for folder in os.listdir(DIRECTORY) if os.path.isdir(os.path.join(DIRECTORY, folder))]
        print("Available categories:", available_categories)
        # Raise an error if the category does not exist
        raise FileNotFoundError(f"Category '{category_path}' not found.")

    # Create the output directory structure if it does not exist
    output_dir = '../output_data'
    landmarks_dir = os.path.join(output_dir, 'landmarks')

    # Ensure output directories exist
    for path in [landmarks_dir]:
        if not os.path.exists(path):
            os.makedirs(path)
            print(f"Created directory: {path}")

    # Specify the output file paths for landmarks and connections
    landmarks = os.path.join(landmarks_dir, f'face_landmarks_{desired_category}.csv')
    
    # Return the paths for further use
    return landmarks, category_path

landmarks_file, category_path = paths(desired_category)

I0000 00:00:1732340530.778154 8271835 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


RuntimeError: Unable to open file at /Users/steven/Desktop/code/py/GCN/GCNN/model/face_landmarker.task

In [22]:
# create CSV
with open(landmarks_file, 'w', newline='') as landmarks_csv:
    
    landmarks_writer = csv.writer(landmarks_csv)
    
    landmarks_writer.writerow(["image_name", "category", "landmark_index", "x", "y", "z"])  # Landmarks
    # create FaceLandmarker
    with FaceLandmarker.create_from_options(options) as landmarker:
        folder = category_path
        print(f"Processing category: {desired_category}")
        
        # Iterate through each image in the specified category
        for image_name in os.listdir(folder):
            image_path = os.path.join(folder, image_name)
            frame = cv2.imread(image_path)
            
            if frame is None:
                print(f"Cannot read image: {image_name}")
                continue

            h, w = frame.shape[:2]
            # to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # create Mediapipe image
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
            
            # FaceLandmarker identify face
            face_landmarker_result = landmarker.detect(mp_image)
            
            # Get face feature
            face_landmarks_list = face_landmarker_result.face_landmarks

            if not face_landmarks_list:
                print(f"No face detected in image: {image_name}")
                continue

            # process face feature
            for face_landmarks in face_landmarks_list:
                # save landmarks
                for idx, landmark in enumerate(face_landmarks):
                    x = landmark.x * w
                    y = landmark.y * h
                    z = landmark.z * w
                    landmarks_writer.writerow([image_name, desired_category, idx, x, y, z])
                
            
            print(f"Processed: {image_name} in category {desired_category}")

I0000 00:00:1732341056.636865 8271835 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1732341056.650428 8271835 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1732341056.675042 8960307 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732341056.686455 8960309 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732341056.734640 8960310 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: S

Processing category: 6
No face detected in image: train_08911_aligned.jpg
Processed: train_06179_aligned.jpg in category 6
Processed: train_01170_aligned.jpg in category 6
Processed: train_03411_aligned.jpg in category 6
Processed: train_01012_aligned.jpg in category 6
No face detected in image: train_07974_aligned.jpg
Processed: train_07002_aligned.jpg in category 6
Processed: train_05318_aligned.jpg in category 6
Processed: train_05151_aligned.jpg in category 6
Processed: train_08803_aligned.jpg in category 6
No face detected in image: train_02597_aligned.jpg
Processed: train_07627_aligned.jpg in category 6
No face detected in image: train_01317_aligned.jpg
Processed: train_06598_aligned.jpg in category 6
No face detected in image: train_06407_aligned.jpg
Processed: train_04810_aligned.jpg in category 6
No face detected in image: train_08750_aligned.jpg
Processed: train_07298_aligned.jpg in category 6
No face detected in image: train_03764_aligned.jpg
Processed: train_08642_aligned.j

In [27]:
landmarks_file = f'../output_data/landmarks/face_landmarks_{desired_category}.csv'
connections_file = f'../output_data/connection.csv'

landmarks_df = pd.read_csv(landmarks_file)
connections_df = pd.read_csv(connections_file)

output_folder = f'../output_data/adjacency/adjacency_{desired_category}'
os.makedirs(output_folder, exist_ok=True)

In [28]:
for image_name in landmarks_df['image_name'].unique():
    # Filter landmarks for the current image
    image_landmarks_df = landmarks_df[landmarks_df['image_name'] == image_name].reset_index(drop=True)
    
    # Extract coordinates and the number of landmarks for the image
    points_coordinates = image_landmarks_df[['x', 'y', 'z']].values
    num_points = len(points_coordinates)
    
    # Initialize the adjacency matrix
    adjacency_matrix = np.zeros((num_points, num_points))

    point_indices_1 = connections_df['point1'].astype(int).values
    point_indices_2 = connections_df['point2'].astype(int).values
    valid_indices = (point_indices_1 < num_points) & (point_indices_2 < num_points)
    point_indices_1 = point_indices_1[valid_indices]
    point_indices_2 = point_indices_2[valid_indices]

    coords1 = points_coordinates[point_indices_1]
    coords2 = points_coordinates[point_indices_2]
    
    # Compute distances for all connection pairs
    distances = np.linalg.norm(coords1 - coords2, axis=1)
    
    # Fill the adjacency matrix with distances
    adjacency_matrix[point_indices_1, point_indices_2] = distances
    adjacency_matrix[point_indices_2, point_indices_1] = distances  # Symmetric
    
    adjacency_df = pd.DataFrame(adjacency_matrix)
    output_path = os.path.join(output_folder, f'adjacency_matrix_{image_name}.csv')
    adjacency_df.to_csv(output_path, index=False)
    print(f"Saved adjacency matrix for {image_name} to {output_path}")

Saved adjacency matrix for train_06179_aligned.jpg to ../output_data/adjacency/adjacency_6/adjacency_matrix_train_06179_aligned.jpg.csv
Saved adjacency matrix for train_01170_aligned.jpg to ../output_data/adjacency/adjacency_6/adjacency_matrix_train_01170_aligned.jpg.csv
Saved adjacency matrix for train_03411_aligned.jpg to ../output_data/adjacency/adjacency_6/adjacency_matrix_train_03411_aligned.jpg.csv
Saved adjacency matrix for train_01012_aligned.jpg to ../output_data/adjacency/adjacency_6/adjacency_matrix_train_01012_aligned.jpg.csv
Saved adjacency matrix for train_07002_aligned.jpg to ../output_data/adjacency/adjacency_6/adjacency_matrix_train_07002_aligned.jpg.csv
Saved adjacency matrix for train_05318_aligned.jpg to ../output_data/adjacency/adjacency_6/adjacency_matrix_train_05318_aligned.jpg.csv
Saved adjacency matrix for train_05151_aligned.jpg to ../output_data/adjacency/adjacency_6/adjacency_matrix_train_05151_aligned.jpg.csv
Saved adjacency matrix for train_08803_aligned.j

# Test

In [20]:
import os

# Define the directories for images and adjacency matrices
IMAGE_DIRECTORY = r"../Image_data/DATASET/train/" + desired_category
ADJACENCY_DIRECTORY = r"output_data/adjacency/adjacency_" + desired_category

def verify_file_pairing(image_dir, adjacency_dir):
    """
    Verify if each adjacency matrix file has a corresponding image file.
    """
    # List all adjacency matrix files
    adjacency_files = [f for f in os.listdir(adjacency_dir) if f.endswith('.csv')]

    unmatched_files = []

    for adjacency_file in adjacency_files:
        # Extract the base name without extensions for matching
        base_name = os.path.splitext(adjacency_file.replace('adjacency_matrix_', '').replace('.csv', ''))[0]

        # Construct the corresponding image file name
        image_file = f"{base_name}.jpg"
        image_path = os.path.join(image_dir, image_file)

        if not os.path.exists(image_path):
            unmatched_files.append((adjacency_file, image_file))

    if unmatched_files:
        print("The following adjacency matrices do not have matching image files:")
        for adj, img in unmatched_files:
            print(f"Adjacency Matrix: {adj} | Expected Image: {img}")
    else:
        print("All adjacency matrices have corresponding image files.")

# Run the verification
verify_file_pairing(IMAGE_DIRECTORY, ADJACENCY_DIRECTORY)


All adjacency matrices have corresponding image files.
